In [21]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all" # cell 的多行输出

新闻种类繁复多样，可以分为财经、体育、科技、娱乐等题材。在本案例中，笔者根据10个关键词从百度新闻爬取了962条新闻（具体方法可以参考本章最后的第2个“补充知识点”），且每个关键词对应的新闻条数相近，现在需要通过Python编程对每条新闻划分类别，匹配到正确的版面，这实际上就是在对新闻进行聚类分群。

In [1]:
import pandas as pd
df=pd.read_excel("./data/新闻.xlsx")
df.head()

,关键词,标题,网址,来源,时间
0,华能信托,信托公司2019年上半年经营业绩概览,http://www.financialnews.com.cn/jrsb_m/xt/zx/2...,中国金融新闻网,2019年07月23日 00:00
1,华能信托,首单信托型企业ABS获批,http://www.jjckb.cn/2018-10/23/c_137552198.htm,经济参考网,2018年10月23日 12:21
2,华能信托,华能贵诚信托孙磊:金融科技助力打造开放信托生态,https://baijiahao.baidu.com/s?id=1639276579449...,同花顺财经,2019年07月17日 10:49
3,华能信托,华能贵诚信托孙磊:金融科技已经成为信托行业重要的基础设施,https://finance.qq.com/a/20190716/007898.htm,腾讯财经,2019年07月16日 18:53
4,华能信托,格力电器股权转让意向方闭门开会 华能信托赫然在列,https://finance.sina.com.cn/trust/roll/2019-05...,新浪,2019年05月22日 22:53


# 新闻文本处理

文本数据，如本节中的新闻标题文本。这种文本类型的数据需要转换为数值类型的数据才能在Python中处理。

这项工作需要用到两个核心技术
- 中文分词
- 文本向量化。

## 中文分词

In [2]:
# jieba分词示例
import jieba
word=jieba.cut('我爱北京天安门')
for i in word:
    print(i)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\26598\AppData\Local\Temp\jieba.cache
Loading model cost 0.502 seconds.
Prefix dict has been built successfully.


我
爱
北京
天安门


In [3]:
# 处理第一个标题
word=jieba.cut(df.iloc[0]['标题'])
result=' '.join(word)
print(result)

信托公司 2019 年 上半年 经营 业绩 概览


In [5]:
# 处理所有标题
words=[]
# iterrows()函数用于遍历 DataFrame 的每一行
for i,row in df.iterrows():
    words.append(' '.join(jieba.cut(row['标题'])))
words[:5]

['信托公司 2019 年 上半年 经营 业绩 概览',
 '首单 信托 型 企业 ABS 获批',
 '华能 贵 诚信 托孙磊 : 金融 科技 助力 打造 开放 信托 生态',
 '华能 贵 诚信 托孙磊 : 金融 科技 已经 成为 信托 行业 重要 的 基础设施',
 '格力电器 股权 转让 意向 方 闭门 开会   华能 信托 赫然 在 列']

## 建立词频矩阵

In [7]:
# CountVectorizer 文本向量化函数
from sklearn.feature_extraction.text import CountVectorizer
test=['金融 科技 厉害', '华能 信托 厉害']
vect=CountVectorizer()
X=vect.fit_transform(test)
X=X.toarray()
X

array([[0, 0, 1, 1, 1],
       [1, 1, 1, 0, 0]], dtype=int64)


CountVectorizer()函数会先根据空格来识别每一句话中的词语

例如，它能从第1条新闻标题中识别出“金融”“科技”“厉害”这3个词，从第2条新闻标题中识别出“华能”“信托”“厉害”这3个词，这2条标题一共能识别出“金融”“科技”“华能”“信托”“厉害”这5个不同的词。

这5个词便构成了这2条新闻标题的词袋，CountVectorizer()函数会自动对词袋中的词进行编号，通过vocabulary_属性便能获取词袋的内容及相应编号

In [8]:
# CountVectorizer()函数最开始是设计用来做英文单词向量化的，所以此处词袋中的词并不是按照拼音首字母进行排序的
vect.vocabulary_

{'金融': 4, '科技': 3, '厉害': 2, '华能': 1, '信托': 0}

标题1为“金融科技厉害”，特征3、4、5对应的词在标题1中各出现1次，上表中在特征3、4、5处对应的值就是1，特征1、2对应的词在标题1中出现0次，对应的值就是0，所以标题1对应的数值数组就是[00111]。

同理，标题2对应的数值数组是[11100]，这样便完成了文本数据的数值化。如果一个词出现了不止一次，例如，在某个标题中“金融”这个词出现了2次，那么其对应的值就是2。

此外，CountVectorizer()函数会自动过滤掉一个字的词，这样会过滤掉“的”“之”等没有重要含义的词，不过同时也会过滤掉“爱”“坑”等可能有重要含义的词，因此，这个特点既是一个优势，也是一个劣势

In [9]:
# 构造特征变量
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
vect=CountVectorizer()
X=vect.fit_transform(words)
X=X.toarray()
# 从此袋中获取不带编号的词
word_bag2=vect.get_feature_names()
df=pd.DataFrame(X,columns=word_bag2)
df.head()

c:\ProgramData\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00700,03,04,08s,09,10,100,11,12,150,...,黄萍,黄金,黑客,黑灰产,黑金,黑马,鼓手,鼻祖,齐聚,龙风
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.shape

(962, 3402)

# 模型训练

## KMeans

In [13]:
from sklearn.cluster import KMeans
kms=KMeans(n_clusters=10,random_state=123)
k_data=kms.fit_predict(df)
k_data[:5]

c:\ProgramData\Miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


array([0, 0, 3, 3, 0])

In [14]:
import numpy as np
word_ary=np.array(words)
word_ary[k_data==1]

array(['... 电视 总台 七夕 特别节目 《 天下 有情人 》 浪漫 升级 , 引领 传统 文化 新 ...',
       '北京 文化 “ 封神 ” : 爆款 如何 持续',
       '深挖 “ 仓颉造 字 ” 历史 文化 , 寿光 这个 村新 时代 文明 实践 有 高招',
       '让 夜间 经济 更 有 文化 味   哪里 才 是 真正 “ 网红 打卡 地 ” ?',
       '嘉兴 : “ 伯鸿 城市 书房 ” 构筑 风雅 桐乡 最美 文化 地标',
       '“ 中华文化 边疆 行 — — 走进 昭通 ” 大型 系列 文化 活动 拉开帷幕',
       '文化 增 活力   旅游 添 魅力   文旅 融合 讓濟南 旅游 悄然 發生 著 變化',
       '华北 、 华东 文化 消费 热情 高   文化 场馆 周边 配套 不足 最 被 诟病',
       '「 地 评线 」 “ 可爱 中国 ” 凝聚 文化 自信', '党建 · 文化 · 环境   锦江区 水井坊 打造 国际化 社区',
       '何 建华 专栏 : 钱汉东 为什么 钟情 追求 “ 最 文化 ”',
       '文化 _   全球 电竞 大会 在 沪 举行 , 启动 首届 “ 上海 电 竞周 ” , 举办 近 百场 ...',
       '昔日 垃圾 地   今朝 文化园 — — 月亮 湾 社区 综合 文化 服务中心 成为 居民 “ ...',
       '讲好 中国 故事   展示 文化 魅力', '【 文化 履游 】 这个 地方 , 文化 与 美食 都 不能 错过',
       '太原 能否 当选 2020 年 “ 东亚 文化 之 都 ” 8 月底 揭晓',
       '2019 首届 明德 儒商 论坛 聚焦 中国 智慧 , 提升 文化 自信',
       '人民日报 海外版 : 抓住 “ 真 文化 ” 而 不是 “ 假 标签 ”', '青春 芒果 节 引领 青年 文化',
       '中国 陶都 ( 宜兴 ) 陶瓷 文化 推介会 在 北京 举行', '文聚邦 — — 文化 产业链 的 生态 服务商',
       '弘扬 传统 婚姻 文化 , 上演 古代 汉服 婚礼 秀 , 执子之手 与尔 偕老',

可以看出 上述基本都与文化相关，分类效果还是可以

## DBSCAN算法

In [16]:
from sklearn.cluster import DBSCAN
# eps参数（画圆半径）为1，min_samples参数（圆内最小样本数）为3
dbs=DBSCAN(eps=1,min_samples=3)
d_data=dbs.fit_predict(df)

In [17]:
d_data

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  1,  2,  3,
        4,  5,  6,  7,  8,  0,  1,  2,  3,  4,  5,  6,  7,  8,  0,  1,  2,
        3,  4,  5,  6,  7,  8,  0,  1,  2,  3,  4,  5,  6,  7,  8,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  0,  1,  2,  3,  4,  5,  6,  7,  8,  0,
        1,  2,  3,  4,  5,  6,  7,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

从图中可以看出，DBSCAN算法对新闻标题的聚类效果较差，其中有大量离群点（-1），即不知道这条新闻标题属于什么分类。

因为进行文本向量化后，每个新闻标题都有3401个特征，过多的特征容易导致样本点间的距离较远，从而产生离群点

因此对于新闻文本而言，KMeans算法的聚类效果很好，DBSCAN算法的聚类效果则不尽如人意，这也说明了对于特征变量较多的数据，KMeans算法的聚类效果要优于DBSCAN算法的聚类效果

# 模型优化

误差主要原因是新闻标题长短不一，在进行中文分词及文本向量化后，长标题和短标题的距离就较远

In [18]:
words_test = ['想去 华能 信托', '华能 信托 很好 想去', '华能 信托 很好 想去 华能 信托 很好 想去']
# 文本向量化
vect=CountVectorizer()
X_test=vect.fit_transform(words_test)
X_test=X_test.toarray()

word_bag2=vect.get_feature_names()
df_test=pd.DataFrame(X_test,columns=word_bag2)
df_test

c:\ProgramData\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,信托,华能,很好,想去
0,1,1,0,1
1,1,1,1,1
2,2,2,2,2


In [22]:
# 计算欧式距离
np.linalg.norm(df_test.iloc[0]-df_test.iloc[1])
np.linalg.norm(df_test.iloc[1]-df_test.iloc[2])

1.0

2.0

In [23]:
# 余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(df_test)

array([[1.       , 0.8660254, 0.8660254],
       [0.8660254, 1.       , 1.       ],
       [0.8660254, 1.       , 1.       ]])

第i行第j列的数字表示第i个数据和第j个数据的余弦相似度。

例如，第2行第3列的数字1是第2条新闻标题和第3条新闻标题的余弦相似度

而第2行第1列的数字0.866则是第2条新闻标题和第1条新闻标题的余弦相似度

左上角至右下角的对角线上的数字都为1，这些数字的意义不大，因为它们表示数据与本身的余弦相似度

In [24]:
# 用余弦相似性 代替 欧氏距离 对模型进行优化
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity=cosine_similarity(df)
from sklearn.cluster import KMeans
kms=KMeans(n_clusters=10,random_state=123)
k_data=kms.fit_predict(cosine_similarity)

c:\ProgramData\Miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
